Building likelihood maps for a set of representative images.

> TODO: use images from dataset and superimpose the bounding boxes on them


In [ ]:
import retinoto_py as fovea
subset_factor = 5
args = fovea.Params(do_fovea=True, batch_size=1, subset_factor=subset_factor)
args

In [ ]:
dataset = 'bbox'
VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, do_full_preprocess=False)
val_loader = fovea.get_loader(args, val_dataset)


In [ ]:
model_filename = args.data_cache /  f'32_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)
model_filename

In [ ]:
# from torchvision.transforms import v2 as transforms
npy_filename_label = args.data_cache / '42_likelihood_maps_label.npy'
npy_filename_max = args.data_cache / '42_likelihood_maps_max.npy'

# %rm {npy_filename}  # FORCING RECOMPUTE

In [ ]:
resolution = (100, 100)
resolution = (30, 30)
resolution = (21, 21)
size_ratio = .4

if npy_filename_label.exists():
    likelihood_maps_label = fovea.np.load(npy_filename_label)
    likelihood_maps_max = fovea.np.load(npy_filename_max)
else:
    n_dataset = len(val_dataset)
    with fovea.torch.no_grad():
        likelihood_maps_label = fovea.np.empty((resolution[0], resolution[1], n_dataset))
        likelihood_maps_max = fovea.np.empty((resolution[0], resolution[1], n_dataset))
        for i_batch, (image, true_idx) in fovea.tqdm(enumerate(val_loader), total=n_dataset):
            image, true_idx = image.to(args.device), true_idx.to(args.device)
            pos_H, pos_W = fovea.get_positions(image.shape[1], image.shape[2], resolution=resolution)
            probas = fovea.compute_likelihood_map(args, model, image.squeeze(0), pos_H, pos_W, size_ratio=size_ratio)
            likelihood_maps_label[:, :, i_batch] = probas[:, true_idx].cpu().numpy().reshape(resolution)
            proba_max, _ = probas.max(axis=-1)
            likelihood_maps_max[:, :, i_batch] = proba_max.cpu().numpy().reshape(resolution)
        fovea.np.save(npy_filename_label, likelihood_maps_label)        
        fovea.np.save(npy_filename_max, likelihood_maps_max)        

In [ ]:
# fig, ax = fovea.plt.subplots()
# contour = ax.contourf(likelihood_maps.mean(axis=-1))
# fig.colorbar(contour, ax=ax)  # Add colorbar
# ax.axis('square')
# fig.set_facecolor(color='white')
likelihood_maps = likelihood_maps_label

In [ ]:
probas.max(axis=-1)

In [ ]:
i_image = 2
proba = likelihood_maps[:, :, i_image]
proba.min(), proba.max()

In [ ]:
fig, ax = fovea.plt.subplots()
contour = ax.contourf(likelihood_maps.mean(axis=-1))
fig.colorbar(contour, ax=ax)  # Add colorbar
ax.axis('square')
fig.set_facecolor(color='white')


In [ ]:
accuracy = likelihood_maps.max(axis=(0, 1))
fig, ax = fovea.plt.subplots()
ax.hist(accuracy)


In [ ]:
import pandas as pd
import numpy as np

pos_H, pos_W = fovea.get_positions(1, 1, resolution=resolution)
pos_H.min(), pos_H.max(), pos_W.min(), pos_W.max(), pos_H.shape, likelihood_maps.shape

In [ ]:
data = []
for do_fovea in [True, False]:    
    likelihood_maps = fovea.np.load(npy_filename)
    data_= pd.DataFrame({
        'horizontal position': pos_H,
        'vertical position': pos_W,
        'probas': likelihood_maps.mean(axis=-1).T.flatten(),
        'retinotopy': 'foveated' if do_fovea else 'uniform' 
    })
    data.append(data_)
combined_data = pd.concat(data, ignore_index=True)
combined_data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
from mpl_toolkits.axes_grid1 import make_axes_locatable
def visualize_likelihood_map(likelihood_map, sigma=0, color='blue', fig=None, axes=None):
    # Lisser la heatmap
    if sigma >0: likelihood_map = gaussian_filter(likelihood_map, sigma=sigma)

    # Calcul des marginales
    marginal_H = likelihood_map.mean(axis=1)  # Marginale verticale
    marginal_W = likelihood_map.mean(axis=0)  # Marginale horizontale

    # Créer la figure et l'axe principal
    fig, ax = plt.subplots(figsize=(10, 8))

    # Afficher la heatmap
    contour = ax.contourf(likelihood_map, levels=20, cmap="viridis")
    fig.colorbar(contour, ax=ax, label='Likelihood')
    # ax.set_title("Carte de probabilité de vraisemblance")
    ax.set_xlabel("Position (W)")
    ax.set_ylabel("Position (H)")

    # Ajouter une grille légère
    ax.grid(alpha=0.3, linestyle='--')

    # Ajouter les lignes centrales (verticale et horizontale)
    mid_w = likelihood_map.shape[1] / 2
    mid_h = likelihood_map.shape[0] / 2
    ax.axvline(x=mid_w, color='white', linestyle='--', linewidth=2)
    ax.axhline(y=mid_h, color='white', linestyle='--', linewidth=2)

    # Ajouter des axes pour les marginales
    divider = make_axes_locatable(ax)

    # Axe pour la marginale horizontale (en haut)
    ax_top = divider.append_axes("top", 1.2, pad=0.1, sharex=ax)
    # ax_top.plot(np.linspace(-1, 1, resolution[1]), marginal_W, color='red')
    ax_top.plot(range(resolution[1]), marginal_W, color=color)
    # ax_top.set_title("Probabilité marginale (axe horizontal)")
    # ax_top.set_ylabel("Probabilité")
    ax_top.set_xticks([])  # Pas de labels sur l'axe x pour éviter la redondance

    # Axe pour la marginale verticale (à droite)
    ax_right = divider.append_axes("right", 1.2, pad=0.1, sharey=ax)
    # ax_right.plot(marginal_H, np.linspace(-1, 1, resolution[0]), color='blue')
    ax_right.plot(marginal_H, range(resolution[0]), color=color)
    # ax_right.set_title("Probabilité marginale (axe vertical)")
    # ax_right.set_xlabel("Probabilité")
    ax_right.set_yticks([])  # Pas de labels sur l'axe y pour éviter la redondance

    plt.tight_layout()
    axes = (ax, ax_top, ax_right)
    return fig, axes

fig, axes = visualize_likelihood_map(likelihood_maps.mean(axis=-1))
plt.show()



In [ ]:
likelihood_maps.shape

In [ ]:

for i_image in [42, 23, 57]:
    fig, axes = visualize_likelihood_map(likelihood_maps[:, :, i_image], sigma=.5)
    plt.show()


In [ ]:
from scipy.optimize import curve_fit

def gaussian_2d(x, y, amplitude, x0, y0, sigma_x, sigma_y, theta):
    """
    2D Gaussian function with rotation (theta).
    """
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    return amplitude * np.exp(-(a*((x-x0)**2) + 2*b*(x-x0)*(y-y0) + c*((y-y0)**2)))

from scipy.optimize import curve_fit

def fit_gaussian(likelihood_map):
    """
    Fit a 2D Gaussian to a likelihood map, constraining x0 and y0 to [-1, 1].
    Returns: amplitude, x0, y0, sigma_x, sigma_y, theta (rotation angle)
    """
    # Create a grid of coordinates
    ny, nx = likelihood_map.shape
    x = np.linspace(-1, 1, nx)
    y = np.linspace(-1, 1, ny)
    x, y = np.meshgrid(x, y)

    # Flatten the likelihood map and coordinates
    data = likelihood_map.flatten()
    x_flat = x.flatten()
    y_flat = y.flatten()

    # Initial guess for parameters
    initial_guess = (
        np.max(likelihood_map),  # amplitude
        0,                       # x0 (middle of the map)
        0,                       # y0
        0.9,                     # sigma_x
        0.9,                     # sigma_y
        0                        # theta (no rotation)
    )

    # Define bounds: (x0, y0) must be in [-1, 1]
    # Format: ([min_amp, min_x0, min_y0, min_sx, min_sy, min_theta], [max_amp, max_x0, max_y0, max_sx, max_sy, max_theta])
    bounds = (
        [-np.inf, -1, -1, 0.01, 0.01, -np.pi],  # Lower bounds
        [np.inf, 1, 1, 2, 2, np.pi]            # Upper bounds
    )

    # Fit the Gaussian with bounds
    try:
        popt, pcov = curve_fit(
            lambda arr, amp, x0, y0, sx, sy, theta:
                gaussian_2d(arr[0], arr[1], amp, x0, y0, sx, sy, theta),
            (x_flat, y_flat),
            data,
            p0=initial_guess,
            bounds=bounds,
            maxfev=100000
        )
        return popt
    except RuntimeError:
        # If fitting fails, return NaN values
        return np.full(6, np.nan)

import pandas as pd

def compute_gaussian_params(likelihood_maps):
    """
    Compute Gaussian parameters for each likelihood map in `likelihood_maps`.
    `likelihood_maps` shape: (height, width, n_images)
    Returns: pandas DataFrame with columns for each parameter
    """
    n_images = likelihood_maps.shape[-1]
    results = {
        'image_index': [],
        'amplitude': [],
        'x0': [],
        'y0': [],
        'sigma_x': [],
        'sigma_y': [],
        'theta': []
    }

    for i_image in fovea.tqdm(range(n_images)):
        likelihood_map = likelihood_maps[:, :, i_image]
        # Smooth the map to reduce noise
        likelihood_map = gaussian_filter(likelihood_map, sigma=1)
        # Fit the Gaussian
        gaussian_params = fit_gaussian(likelihood_map)

        # Append results to the dictionary
        results['image_index'].append(i_image)
        results['amplitude'].append(gaussian_params[0])
        results['x0'].append(gaussian_params[1])
        results['y0'].append(gaussian_params[2])
        results['sigma_x'].append(gaussian_params[3])
        results['sigma_y'].append(gaussian_params[4])
        results['theta'].append(gaussian_params[5])

    # Convert the dictionary to a pandas DataFrame
    df = pd.DataFrame(results)
    return df


# Example usage:
gaussian_df = compute_gaussian_params(likelihood_maps)
gaussian_df.head()

In [ ]:
# Joint KDE plot with marginals
g = sns.jointplot(
    data=gaussian_df,
    x="x0",
    y="y0",
    kind="kde",
    cmap="viridis",
)
plt.show()

In [ ]:
# Example: Check if sigma_x is usually bigger than sigma_y
is_sigma_x_bigger = gaussian_df["sigma_x"] > gaussian_df["sigma_y"]
print(f"Fraction of images where sigma_x > sigma_y (more horizontal): {is_sigma_x_bigger.mean():.2f}")
print(f"Fraction of images where sigma_y > sigma_x (more vertical): {(~is_sigma_x_bigger).mean():.2f}")

In [ ]:
import numpy as np
import lmfit
from lmfit import Parameters, minimize, report_fit

def gaussian_2d_lmfit(params, x, y, data):
    """
    2D Gaussian function for lmfit.
    """
    amp = params['amplitude']
    x0 = params['x0']
    y0 = params['y0']
    sx = params['sigma_x']
    sy = params['sigma_y']
    theta = params['theta']

    a = (np.cos(theta)**2)/(2*sx**2) + (np.sin(theta)**2)/(2*sy**2)
    b = -(np.sin(2*theta))/(4*sx**2) + (np.sin(2*theta))/(4*sy**2)
    c = (np.sin(theta)**2)/(2*sx**2) + (np.cos(theta)**2)/(2*sy**2)

    model = amp * np.exp(-(a*((x-x0)**2) + 2*b*(x-x0)*(y-y0) + c*((y-y0)**2)))
    return model - data  # Residuals for least-squares minimization

def fit_gaussian_lmfit(likelihood_map):
    """
    Fit a 2D Gaussian to a likelihood map using lmfit, constraining x0 and y0 to [-1, 1].
    Returns: amplitude, x0, y0, sigma_x, sigma_y, theta (rotation angle)
    """
    # Create a grid of coordinates
    ny, nx = likelihood_map.shape
    x = np.linspace(-1, 1, nx)
    y = np.linspace(-1, 1, ny)
    x, y = np.meshgrid(x, y)

    # Flatten the likelihood map and coordinates
    data = likelihood_map.flatten()
    x_flat = x.flatten()
    y_flat = y.flatten()

    # Create Parameters object with initial guesses and bounds
    params = lmfit.Parameters()
    params.add('amplitude', value=np.max(likelihood_map), min=0)  # Amplitude must be positive
    params.add('x0', value=0, min=-1, max=1)  # x0 in [-1, 1]
    params.add('y0', value=0, min=-1, max=1)  # y0 in [-1, 1]
    params.add('sigma_x', value=0.5, min=0.01, max=4)  # sigma_x bounds
    params.add('sigma_y', value=0.5, min=0.01, max=4)  # sigma_y bounds
    params.add('theta', value=0, min=-np.pi/2, max=np.pi/2)  # theta bounds

    # Minimize the residuals using least-squares
    result = minimize(
        gaussian_2d_lmfit,
        params,
        args=(x_flat, y_flat, data),
        # method='leastsq',  # Levenberg-Marquardt algorithm
        method='lbfgsb',
        tol=1e-8,
        # xtol=1e-8,
        max_nfev=20000
    )

    # Check if the fit succeeded
    if not result.success:
        return np.full(6, np.nan)

    fit_result = {}
    for name, param in result.params.items():
        fit_result[name] = param.value
        # fit_result[f"{name}_error"] = param.stderr

    return fit_result


def compute_gaussian_params(likelihood_maps, sigma=.5):
    """
    Compute Gaussian parameters for each likelihood map in `likelihood_maps`.
    `likelihood_maps` shape: (height, width, n_images)
    Returns: pandas DataFrame with columns for each parameter
    """
    n_images = likelihood_maps.shape[-1]
    results = []

    for i_image in fovea.tqdm(range(n_images)):
        likelihood_map = likelihood_maps[:, :, i_image]
        # Smooth the map to reduce noise
        likelihood_map = gaussian_filter(likelihood_map, sigma=sigma)
        # Fit the Gaussian
        results.append(fit_gaussian_lmfit(likelihood_map))


    # Convert the dictionary to a pandas DataFrame
    gaussian_df = pd.DataFrame(results)
    gaussian_df["theta_deg"] = np.rad2deg(gaussian_df["theta"])
    gaussian_df['sigma_major'] = np.where(gaussian_df['sigma_x'] > gaussian_df['sigma_y'], 
                                          gaussian_df['sigma_x'], gaussian_df['sigma_y'])
    gaussian_df['sigma_minor'] = np.where(gaussian_df['sigma_x'] < gaussian_df['sigma_y'], 
                                          gaussian_df['sigma_x'], gaussian_df['sigma_y'])

    gaussian_df["elongation"] = gaussian_df['sigma_major'] / gaussian_df['sigma_minor']
    return gaussian_df
gaussian_df = compute_gaussian_params(likelihood_maps, sigma=.5)
gaussian_df.head()

In [ ]:
# Joint KDE plot with marginals
g = sns.jointplot(
    data=gaussian_df,
    x="x0",
    y="y0",
    kind="kde",
    cmap="viridis",
)
plt.show()

In [ ]:
# Joint KDE plot with marginals
g = sns.jointplot(
    data=gaussian_df,
    x="sigma_x",
    y="sigma_y",
    kind="scatter",
    cmap="viridis",
)
plt.show()

In [ ]:
# Joint KDE plot with marginals
g = sns.jointplot(
    data=gaussian_df,
    x="sigma_minor",
    y="sigma_major",
    kind="scatter",
    alpha=.2,
)
plt.show()

In [ ]:
sns.histplot(
    data=gaussian_df,
    x='elongation',
    bins=40,
    element='step',  # Use step plots for clarity
    kde=False,  # Disable KDE for log-scaled x-axis
    color='skyblue',
    alpha=0.7,
    edgecolor='black',
    log_scale=(True, False)  # Log scale only for the x-axis
)

# Compute the peak (bin with the highest count)
counts, bin_edges = np.histogram(np.log(gaussian_df['elongation']), bins=40)
peak_bin_index = np.argmax(counts)
log_peak_value = (bin_edges[peak_bin_index] + bin_edges[peak_bin_index + 1]) / 2  # Midpoint of the peak bin
peak_count = counts[peak_bin_index]
peak_value = np.exp(log_peak_value)

# Annotate the peak on the plot
plt.axvline(x=peak_value, color='red', linestyle='--', label=f'Peak: {peak_value:.2f}')
plt.text(
    4,
    120,
    f' Peak @ {peak_value:.2f}',
    ha='center',
    va='bottom',
    color='red',
    fontsize=24,
    bbox=dict(facecolor='white', alpha=0., edgecolor='none')
)

In [ ]:
gaussian_df['elongation'].min(), gaussian_df['elongation'].max()

In [ ]:
sns.histplot(
    data=gaussian_df,
    x='theta_deg',
    bins=30,
    element='step'  # Use step plots for clarity
)


Interpretation:

* θ=0: The major axis is aligned with the x-axis.
* θ=π/2: The major axis is aligned with the y-axis.
* θ=π/4: The major axis is at a 45° angle.
